In [ ]:
#pip install nbimporter

In [74]:
import warnings
warnings.filterwarnings('ignore')

In [75]:
import pandas as pd
import numpy as np
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
import re

In [76]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="obras_vzla")
import time 

In [77]:
#driver configuration
opciones=Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

### Para comenzar con el scrapping, importamos las funciones que tenemos en el archivo 'procesos'

In [78]:
from procesos import *

### Establecemos el nombre de las columnas que tendrá el dataframe

In [79]:
columnas = ['Obra','Status','Ejecución física %','Ejecución financiera %','Estado','Municipio','Dirección','Personas afectadas','Ente responsable','Funcionario responsable','Empresa contratista','Valor total de la obra USD (millones)','Obra financiada por FONDEN','Monto comprometido USD (millones)','Fecha estimada de inicio','Fecha real de inicio','Fecha culminación estimada de la obra','Fecha de paralización']

### Llamamos a la función 'obtener_datos_obra' que realiza el proceso de web scraping (selenium), obtenemos los datos que queremos de la obra y lo convertimos en un dataframe

In [80]:
url = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/complejo-habitacional-che-guevara/'
complejo_habitacional = obtener_datos_obra(url)

In [81]:
df = pd.DataFrame(complejo_habitacional)
df = df.T

df['Obra financiada por FONDEN'] = 'S/I'
df['Monto comprometido'] = 'S/I'

#Ordenamos las columnas
df = df[[0,1,2,3,4,5,6,7,8,9,10,11,'Obra financiada por FONDEN','Monto comprometido',12,13,14,15]]
df.columns = columnas
df

,Obra,Status,Ejecución física %,Ejecución financiera %,Estado,Municipio,Dirección,Personas afectadas,Ente responsable,Funcionario responsable,Empresa contratista,Valor total de la obra USD (millones),Obra financiada por FONDEN,Monto comprometido USD (millones),Fecha estimada de inicio,Fecha real de inicio,Fecha culminación estimada de la obra,Fecha de paralización
0,COMPLEJO HABITACIONAL «CHE GUEVARA»,Con recursos desembolsados - no iniciada,S/I,S/I,Estado Yaracuy,Independencia,"Sector San Rafael, Independencia",Entre 0-5.000,Gran Misión Vivienda Venezuela (GMVV),S/I,S/I,USD 3.174.603,S/I,S/I,Noviembre 2014,S/I,Marzo 2016,S/I


### Vamos a realizar el mismo procedimiento para las demás obras que tenemos en nuestra página web que estamos scrapeando

In [82]:
url2 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/desarrollo-habitacional-baru-i-baru-ii-y-baru-iii/'
complejo_baru = obtener_datos_obra(url2)

### Observamos que este dato no posee el mismo formato que el dataframe anterior, por lo que eliminamos, agregamos y renombramos columnas para que coincidan todos los datos en el dataframe final

In [83]:
df2 = pd.DataFrame(complejo_baru)

In [84]:
df2 = df2.T #intercambiamos las filas y las columnas para tener la disposición de los datos como deseamos
df2 = df2.drop(df2.columns[[2, 3]], axis=1) #eliminamos las columnas 2 y 3 que no tienen información

df2.columns = columnas
df2

,Obra,Status,Ejecución física %,Ejecución financiera %,Estado,Municipio,Dirección,Personas afectadas,Ente responsable,Funcionario responsable,Empresa contratista,Valor total de la obra USD (millones),Obra financiada por FONDEN,Monto comprometido USD (millones),Fecha estimada de inicio,Fecha real de inicio,Fecha culminación estimada de la obra,Fecha de paralización
0,"DESARROLLO HABITACIONAL BARU I, BARU II Y BARU...",Paralizada,"14,53%",S/I,Estado Zulia,San Francisco,"Av. Don Manuel Belloso, Base Aérea Rafael Urda...",Entre 0-5.000,Ministerio de Defensa,S/I,S/I,USD 25.636.094,Si,"USD 10.069.663,10",30/8/2007,S/I,31/3/2015,S/I


In [85]:
url3 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/desarrollo-habitacional-nueva-ojeda/'
nueva_ojeda = obtener_datos_obra(url3)

In [86]:
df3 = pd.DataFrame(nueva_ojeda)
df3 = df3.T

df3 = df3.drop(df3.columns[[2, 3]], axis=1) #eliminamos las columnas 2 y 3 que no tienen información
df3.columns = columnas
df3

,Obra,Status,Ejecución física %,Ejecución financiera %,Estado,Municipio,Dirección,Personas afectadas,Ente responsable,Funcionario responsable,Empresa contratista,Valor total de la obra USD (millones),Obra financiada por FONDEN,Monto comprometido USD (millones),Fecha estimada de inicio,Fecha real de inicio,Fecha culminación estimada de la obra,Fecha de paralización
0,DESARROLLO HABITACIONAL NUEVA OJEDA,Paralizada,70%,S/I,Estado Zulia,Lagunillas,"Carretera U, sector El Menito. Ciudad Ojeda.",Entre 0-5.000,Gran Misión Vivienda Venezuela (GMVV) / Misión...,S/I,"Iranian International Housing Company, C. A.",USD 518.550.100,Si,"USD 166.143.982,21",28/08/2009,S/I,30/07/2014,S/I


In [87]:
url4 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/urbanizacion-banco-obrero/'
banco_obrero = obtener_datos_obra(url4)

In [88]:
df4 = pd.DataFrame(banco_obrero)
df4 = df4.T

df4 = df4.drop(df4.columns[[2, 3]], axis=1) #eliminamos las columnas 2 y 3 que no tienen información

df4['Obra financiada por FONDEN'] = 'S/I'  #agregamos las columnas para que todo el formato se mantenga
df4['Monto comprometido'] = 'S/I'

#Ordenamos las columnas
df4 = df4[[0,1,4,5,6,7,8,9,10,11,12,13,'Obra financiada por FONDEN','Monto comprometido',14,15,16,17]]
df4.columns = columnas

df4

,Obra,Status,Ejecución física %,Ejecución financiera %,Estado,Municipio,Dirección,Personas afectadas,Ente responsable,Funcionario responsable,Empresa contratista,Valor total de la obra USD (millones),Obra financiada por FONDEN,Monto comprometido USD (millones),Fecha estimada de inicio,Fecha real de inicio,Fecha culminación estimada de la obra,Fecha de paralización
0,URBANIZACIÓN BANCO OBRERO,Paralizada,15%,S/I,Estado Yaracuy,San Felipe,San Felipe,Entre 0-5.000,Gran Misión Vivienda Venezuela (GMVV),S/I,S/I,USD 28.571.429,S/I,S/I,Noviembre 2014,2015,2015,2015


In [89]:
url5 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/conjunto-de-residencias-av-cedeno/'
av_cedeno = obtener_datos_obra(url5)

In [90]:
df5 = pd.DataFrame(av_cedeno)
df5 = df5.T

df5['Obra financiada por FONDEN'] = 'S/I'
df5['Monto comprometido'] = 'S/I'

#Ordenamos las columnas
df5 = df5[[0,1,2,3,4,5,6,7,8,9,10,11,'Obra financiada por FONDEN','Monto comprometido',12,13,14,15]]
df5.columns = columnas


In [91]:
url6 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/construccion-urbanismo-che-guevara-de-la-mision-vivienda/'
che_guevara = obtener_datos_obra(url6)

In [92]:
df6 = pd.DataFrame(che_guevara)
df6 = df6.T

df6 = df6.drop(df6.columns[[2, 3]], axis=1) #eliminamos las columnas 2 y 3 que no tienen información

df6['Obra financiada por FONDEN'] = 'S/I'
df6['Monto comprometido'] = 'S/I'

df6 = df6[[0,1,4,5,6,7,8,9,10,11,12,13,'Obra financiada por FONDEN','Monto comprometido',14,15,16,17]]
df6.columns = columnas

df6

,Obra,Status,Ejecución física %,Ejecución financiera %,Estado,Municipio,Dirección,Personas afectadas,Ente responsable,Funcionario responsable,Empresa contratista,Valor total de la obra USD (millones),Obra financiada por FONDEN,Monto comprometido USD (millones),Fecha estimada de inicio,Fecha real de inicio,Fecha culminación estimada de la obra,Fecha de paralización
0,CONSTRUCCIÓN URBANISMO CHE GUEVARA DE LA MISIÓ...,Paralizada,30%,S/I,Estado Lara,Palavecino,Parroquia José Gregorio Bastidas,Entre 0-5.000,"Corporación de Desarrollo ""Jacinto Lara"" (CORP...",Luis Reyes Reyes,S/I,USD 46.766.793,S/I,S/I,2014,2014,2016,S/I


### Procedemos a juntar todos los dataframes

In [119]:
# Concatena los DataFrames en uno solo
data_viviendas = pd.concat([df,df2,df3,df4,df5,df6],ignore_index=True)

In [120]:
data_viviendas

,Obra,Status,Ejecución física %,Ejecución financiera %,Estado,Municipio,Dirección,Personas afectadas,Ente responsable,Funcionario responsable,Empresa contratista,Valor total de la obra USD (millones),Obra financiada por FONDEN,Monto comprometido USD (millones),Fecha estimada de inicio,Fecha real de inicio,Fecha culminación estimada de la obra,Fecha de paralización
0,COMPLEJO HABITACIONAL «CHE GUEVARA»,Con recursos desembolsados - no iniciada,S/I,S/I,Estado Yaracuy,Independencia,"Sector San Rafael, Independencia",Entre 0-5.000,Gran Misión Vivienda Venezuela (GMVV),S/I,S/I,USD 3.174.603,S/I,S/I,Noviembre 2014,S/I,Marzo 2016,S/I
1,"DESARROLLO HABITACIONAL BARU I, BARU II Y BARU...",Paralizada,"14,53%",S/I,Estado Zulia,San Francisco,"Av. Don Manuel Belloso, Base Aérea Rafael Urda...",Entre 0-5.000,Ministerio de Defensa,S/I,S/I,USD 25.636.094,Si,"USD 10.069.663,10",30/8/2007,S/I,31/3/2015,S/I
2,DESARROLLO HABITACIONAL NUEVA OJEDA,Paralizada,70%,S/I,Estado Zulia,Lagunillas,"Carretera U, sector El Menito. Ciudad Ojeda.",Entre 0-5.000,Gran Misión Vivienda Venezuela (GMVV) / Misión...,S/I,"Iranian International Housing Company, C. A.",USD 518.550.100,Si,"USD 166.143.982,21",28/08/2009,S/I,30/07/2014,S/I
3,URBANIZACIÓN BANCO OBRERO,Paralizada,15%,S/I,Estado Yaracuy,San Felipe,San Felipe,Entre 0-5.000,Gran Misión Vivienda Venezuela (GMVV),S/I,S/I,USD 28.571.429,S/I,S/I,Noviembre 2014,2015,2015,2015
4,CONJUNTO DE RESIDENCIAS AV. CEDEÑO,Paralizada,S/I,S/I,Estado Yaracuy,San Felipe,"Av. Cedeño, sector Ruiz Pineda Sur.",Entre 0-5.000,Gran Misión Vivienda Venezuela (GMVV),S/I,S/I,USD 21.427.472,S/I,S/I,Noviembre 2015,S/I,Junio 2016,S/I
5,CONSTRUCCIÓN URBANISMO CHE GUEVARA DE LA MISIÓ...,Paralizada,30%,S/I,Estado Lara,Palavecino,Parroquia José Gregorio Bastidas,Entre 0-5.000,"Corporación de Desarrollo ""Jacinto Lara"" (CORP...",Luis Reyes Reyes,S/I,USD 46.766.793,S/I,S/I,2014,2014,2016,S/I


# Para conocer las coordenadas de cada una de las obras, hacemos uso de Geopy

In [121]:
#Asignamos como dirección el Municipio y Estado de cada obra para conocer sus coordenadas
municipio_estado = data_viviendas['Municipio'] + ', ' + data_viviendas['Estado']
lista_municipio_estado = municipio_estado.tolist()

In [122]:
latitudes = obtener_latitudes(lista_municipio_estado) #obtenemos la latitud de la obra

In [123]:
longitudes = obtener_longitudes(lista_municipio_estado) #obtenemos la longitud de la obra

In [124]:
data_viviendas['Latitud'] = latitudes #agregamos una nueva columna con todos los valores de las latitudes
data_viviendas['Longitud'] = longitudes #agregamos una nueva columna con todos los valores de las longitudes

### Extraemos la parte numérica de la columna 'Valor total de la obra' para tener un dato de tipo entero

In [125]:
def extraer_parte_numerica(valor):
    valor_numerico = valor.replace('USD ', '').replace('.', '')  # Elimina 'USD ' y los puntos
    return int(valor_numerico)

In [126]:
# Aplicar la función a la columna 'Valor total de la obra'
data_viviendas['Valor total de la obra USD (millones)'] = data_viviendas['Valor total de la obra USD (millones)'].apply(extraer_parte_numerica)

#Dividimos para tener el total de millones en un número mas legible
data_viviendas['Valor total de la obra USD (millones)'] = (data_viviendas['Valor total de la obra USD (millones)'] / 1000000).astype(float)


### Limpiamos la columna 'Monto comprometido', extraemos la parte numérica, llenamos los valores nulos con '0', transformamos a un tipo de dato float

In [129]:
data_viviendas['Monto comprometido USD (millones)'] = data_viviendas['Monto comprometido USD (millones)'].apply(extraer_parte_numerica)

ValueError: invalid literal for int() with base 10: 'S/I'

In [ ]:
data_final['Monto comprometido USD (millones)'] = data_final['Monto comprometido USD (millones)'].fillna('0')

# Convertir la columna a cadena, eliminar los puntos y reemplazar las comas por puntos
data_final['Monto comprometido USD (millones)'] = data_final['Monto comprometido USD (millones)'].astype(str)
data_final['Monto comprometido USD (millones)'] = data_final['Monto comprometido USD (millones)'].str.replace('.', '').str.replace(',', '.')

# Convertir la columna a tipo float
data_final['Monto comprometido USD (millones)'] = pd.to_numeric(data_final['Monto comprometido USD (millones)'], errors='coerce')

#Dividimos para tener el total de millones en un número más legible
data_final['Monto comprometido USD (millones)'] = (data_final['Monto comprometido USD (millones)'] / 1000000).astype(int)

### Cambiamos el tipo de dato de la columna 'Fecha estimada de inicio' por datetime y extraemos el año

In [ ]:
# Convertir la columna 'Fecha estimada de inicio' a datetime
data_final['Fecha estimada de inicio'] = pd.to_datetime(data_final['Fecha estimada de inicio'], errors='coerce')
data_final['Fecha estimada de inicio'] = data_final['Fecha estimada de inicio'].dt.year

### Transformamos las columnas 'Ejecución física' y 'Ejecución financiera' en valores de tipo float, para ello llenamos los valores 'S/I' con 0 y extraemos la parte numérica 

In [ ]:
data_final['Ejecución física %'] = data_final['Ejecución física %'].replace('S/I', np.nan)
data_final['Ejecución financiera %'] = data_final['Ejecución financiera %'].replace('S/I', np.nan)

In [ ]:
data_final['Ejecución física %'] = data_final['Ejecución física %'].apply(extract_numeric_value)
data_final['Ejecución física %'] = data_final['Ejecución física %'].str.replace(',', '.').replace({None: '0'})

# Convertir la columna a tipo float
data_final['Ejecución física %'] = data_final['Ejecución física %'].astype(float)

In [ ]:
data_final['Ejecución financiera %'] = data_final['Ejecución financiera %'].apply(extract_numeric_value)
data_final['Ejecución financiera %'] = data_final['Ejecución financiera %'].str.replace(',', '.').replace({None: '0'})

# Convertir la columna a tipo float
data_final['Ejecución financiera %'] = data_final['Ejecución financiera %'].astype(float)

### Añadimos una nueva columna para indicar cuál es el sector correspondiente a la obra

In [ ]:
data_final['Sector'] = 'Transporte'

In [ ]:
#Reordenamos las columnas
nuevo_orden = ['Sector','Obra','Status','Ejecución física %','Ejecución financiera %','Estado','Municipio','Dirección','Personas afectadas','Ente responsable','Funcionario responsable','Empresa contratista','Valor total de la obra USD (millones)','Obra financiada por FONDEN','Monto comprometido USD (millones)','Fecha estimada de inicio','Fecha real de inicio','Fecha culminación estimada de la obra','Fecha de paralización', 'Latitud', 'Longitud']
data_final = data_final[nuevo_orden]
data_final.head(20)

### Exportamos nuestro dataframe 

In [ ]:
data_final.to_csv('obras_transporte.csv', index=False)